# Semantic Kernel Tool Use Example

This document provides an overview and explanation of the code used to create a Semantic Kernel-based tool that integrates with Azure AI Search for Retrieval-Augmented Generation (RAG). The example demonstrates how to build an AI agent that retrieves travel documents from an Azure AI Search index, augments user queries with semantic search results, and streams detailed travel recommendations.

## Initializing the Environment

### Importing Packages
The following code imports the necessary packages:

In [ ]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### Creating the Semantic Kernel and AI Service

A Semantic Kernel instance is created and configured with an asynchronous OpenAI chat completion service. The service is added to the kernel for use in generating responses.

In [ ]:
load_dotenv()
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### Defining the Prompt Plugin

The PromptPlugin is a native plugin that defines a function to build an augmented prompt using retrieval context

In [ ]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

## Vector Database Initialization

We initialize Azure AI Search with persistent storage and add enhanced sample documents. Azure AI Search will be used to store and retrieve documents that provide context for generating accurate responses.

In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Check if index already exists if not, create it
try:
    existing_index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    # Create the index if it doesn't exist
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)


# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."},
    {"id": "6", "content": "Experience the beauty of the Maldives with our all-inclusive resort packages."},
    {"id": "7", "content": "Explore the Swiss Alps with our guided hiking and skiing tours."},
    {"id": "8", "content": "Discover the wildlife of Africa with our safari adventure packages."},
    {"id": "9", "content": "Contoso Travel's customer service team is available 24/7 to assist you."},
    {"id": "10", "content": "Book your dream vacation with Contoso Travel today!"},
    {"id": "11", "content": "Contoso Travel offers a wide range of travel packages for every budget."},
    {"id": "12", "content": "Our travel experts are here to help you plan the perfect getaway."},
    {"id": "13", "content": "Contoso Travel is committed to providing exceptional service and unforgettable experiences."},
    {"id": "14", "content": "Join us for exclusive travel deals and promotions."},
    {"id": "15", "content": "Contoso Travel is your trusted partner for all your travel needs."},
    {"id": "16", "content": "Experience the world with Contoso Travel's curated travel experiences."},
    {"id": "17", "content": "Contoso Travel offers eco-friendly travel options for the environmentally conscious traveler."},
    {"id": "18", "content": "Our travel packages include everything from flights to accommodations and activities."},
    {"id": "19", "content": "Contoso Travel is dedicated to making your travel dreams a reality."},
    {"id": "20", "content": "Explore new cultures and cuisines with Contoso Travel's guided tours."},
    {"id": "21", "content": "Contoso Travel offers travel packages for families, couples, and solo travelers."},
    {"id": "22", "content": "Our travel advisors are experts in creating personalized travel experiences."},
    {"id": "23", "content": "Contoso Travel is proud to partner with local businesses and communities."},
    {"id": "24", "content": "Join our loyalty program for exclusive discounts and rewards."},
    {"id": "25", "content": "Contoso Travel is your gateway to the world's most beautiful destinations."},
    {"id": "26", "content": "Experience the thrill of adventure travel with Contoso Travel."},
    {"id": "27", "content": "Contoso Travel offers travel packages for every type of traveler."},
    {"id": "28", "content": "Our travel experts are here to help you plan the perfect vacation."},
    {"id": "29", "content": "Contoso Travel is committed to providing exceptional service and unforgettable experiences."},
    {"id": "30", "content": "Join us for exclusive travel deals and promotions."},
    {"id": "31", "content": "Contoso Travel is your trusted partner for all your travel needs."},
    {"id": "32", "content": "Experience the world with Contoso Travel's curated travel experiences."},
    {"id": "33", "content": "Contoso Travel offers eco-friendly travel options for the environmentally conscious traveler."},
    {"id": "34", "content": "Our travel packages include everything from flights to accommodations and activities."},
    {"id": "35", "content": "Contoso Travel is dedicated to making your travel dreams a reality."},
    {"id": "36", "content": "Explore new cultures and cuisines with Contoso Travel's guided tours."},
    {"id": "37", "content": "Contoso Travel offers travel packages for families, couples, and solo travelers."},
    {"id": "38", "content": "Our travel advisors are experts in creating personalized travel experiences."},
    {"id": "39", "content": "Contoso Travel is proud to partner with local businesses and communities."},
    {"id": "40", "content": "Join our loyalty program for exclusive discounts and rewards."},
    {"id": "41", "content": "Contoso Travel is your gateway to the world's most beautiful destinations."},
    {"id": "42", "content": "Experience the thrill of adventure travel with Contoso Travel."},
    {"id": "43", "content": "Contoso Travel offers travel packages for every type of traveler."},
    {"id": "44", "content": "Our travel experts are here to help you plan the perfect vacation."},
    {"id": "45", "content": "Contoso Travel is committed to providing exceptional service and unforgettable experiences."},
    {"id": "46", "content": "Join us for exclusive travel deals and promotions."},
    {"id": "47", "content": "Contoso Travel is your trusted partner for all your travel needs."},
    {"id": "48", "content": "Experience the world with Contoso Travel's curated travel experiences."},
    {"id": "49", "content": "Contoso Travel offers eco-friendly travel options for the environmentally conscious traveler."},
    {"id": "50", "content": "Our travel packages include everything from flights to accommodations and activities."},
    {"id": "51", "content": "Contoso Travel is dedicated to making your travel dreams a reality."},
    {"id": "52", "content": "Explore new cultures and cuisines with Contoso Travel's guided tours."},
    {"id": "53", "content": "Contoso Travel offers travel packages for families, couples, and solo travelers."},
    {"id": "54", "content": "Our travel advisors are experts in creating personalized travel experiences."},
    {"id": "55", "content": "Contoso Travel is proud to partner with local businesses and communities."},
    {"id": "56", "content": "Join our loyalty program for exclusive discounts and rewards."},
    {"id": "57", "content": "Contoso Travel is your gateway to the world's most beautiful destinations."},
    {"id": "58", "content": "Experience the thrill of adventure travel with Contoso Travel."},
    {"id": "59", "content": "Contoso Travel offers travel packages for every type of traveler."},
    {"id": "60", "content": "Our travel experts are here to help you plan the perfect vacation."},
    {"id": "61", "content": "Contoso Travel is committed to providing exceptional service and unforgettable experiences."},
    {"id": "62", "content": "Join us for exclusive travel deals and promotions."},
    {"id": "63", "content": "Contoso Travel is your trusted partner for all your travel needs."},
    {"id": "64", "content": "Experience the world with Contoso Travel's curated travel experiences."},
    {"id": "65", "content": "Contoso Travel offers eco-friendly travel options for the environmentally conscious traveler."},
    {"id": "66", "content": "Our travel packages include everything from flights to accommodations and activities."},
    {"id": "67", "content": "Contoso Travel is dedicated to making your travel dreams a reality."},
    {"id": "68", "content": "Explore new cultures and cuisines with Contoso Travel's guided tours."},
    {"id": "69", "content": "Contoso Travel offers travel packages for families, couples, and solo travelers."},
    {"id": "70", "content": "Our travel advisors are experts in creating personalized travel experiences."},
    {"id": "71", "content": "Contoso Travel is proud to partner with local businesses and communities."},
    {"id": "72", "content": "Join our loyalty program for exclusive discounts and rewards."},
    {"id": "73", "content": "Contoso Travel is your gateway to the world's most beautiful destinations."},
    {"id": "74", "content": "Experience the thrill of adventure travel with Contoso Travel."},
    {"id": "75", "content": "Contoso Travel offers travel packages for every type of traveler."},
    {"id": "76", "content": "Our travel experts are here to help you plan the perfect vacation."},
    {"id": "77", "content": "Contoso Travel is committed to providing exceptional service and unforgettable experiences."},
    {"id": "78", "content": "Join us for exclusive travel deals and promotions."},
    {"id": "79", "content": "Contoso Travel is your trusted partner for all your travel needs."},
    {"id": "80", "content": "Experience the world with Contoso Travel's curated travel experiences."},
    {"id": "81", "content": "Contoso Travel offers eco-friendly travel options for the environmentally conscious traveler."},
    {"id": "82", "content": "Our travel packages include everything from flights to accommodations and activities."},
    {"id": "83", "content": "Contoso Travel is dedicated to making your travel dreams a reality."},
    {"id": "84", "content": "Explore new cultures and cuisines with Contoso Travel's guided tours."},
    {"id": "85", "content": "Contoso Travel offers travel packages for families, couples, and solo travelers."},
    {"id": "86", "content": "Our travel advisors are experts in creating personalized travel experiences."},
    {"id": "87", "content": "Contoso Travel is proud to partner with local businesses and communities."},
    {"id": "88", "content": "Join our loyalty program for exclusive discounts and rewards."},
    {"id": "89", "content": "Contoso Travel is your gateway to the world's most beautiful destinations."},
    {"id": "90", "content": "Experience the thrill of adventure travel with Contoso Travel."},
    {"id": "91", "content": "Contoso Travel offers travel packages for every type of traveler."},
    {"id": "92", "content": "Our travel experts are here to help you plan the perfect vacation."},
    {"id": "93", "content": "Contoso Travel is committed to providing exceptional service and unforgettable experiences."},
    {"id": "94", "content": "Join us for exclusive travel deals and promotions."},
    {"id": "95", "content": "Contoso Travel is your trusted partner for all your travel needs."},
    {"id": "96", "content": "Experience the world with Contoso Travel's curated travel experiences."},
    {"id": "97", "content": "Contoso Travel offers eco-friendly travel options for the environmentally conscious traveler."},
    {"id": "98", "content": "Our travel packages include everything from flights to accommodations and activities."},
    {"id": "99", "content": "Contoso Travel is dedicated to making your travel dreams a reality."},
    {"id": "100", "content": "Explore new cultures and cuisines with Contoso Travel's guided tours."},
    {"id": "101", "content": "Contoso Travel offers travel packages for families, couples, and solo travelers."},
    {"id": "102", "content": "Our travel advisors are experts in creating personalized travel experiences."},
    {"id": "103", "content": "Contoso Travel is proud to partner with local businesses and communities."},
    {"id": "104", "content": "Join our loyalty program for exclusive discounts and rewards."},
    {"id": "105", "content": "Contoso Travel is your gateway to the world's most beautiful destinations."},
    {"id": "106", "content": "Experience the thrill of adventure travel with Contoso Travel."},
    {"id": "107", "content": "Contoso Travel offers travel packages for every type of traveler."},
    {"id": "108", "content": "Our travel experts are here to help you plan the perfect vacation."},
    {"id": "109", "content": "Contoso Travel is committed to providing exceptional service and unforgettable experiences."},
    {"id": "110", "content": "Join us for exclusive travel deals and promotions."},
    {"id": "111", "content": "Contoso Travel is your trusted partner for all your travel needs."},
    {"id": "112", "content": "Experience the world with Contoso Travel's curated travel experiences."},
    {"id": "113", "content": "Contoso Travel offers eco-friendly travel options for the environmentally conscious traveler."},
    {"id": "114", "content": "Our travel packages include everything from flights to accommodations and activities."},
    {"id": "115", "content": "Contoso Travel is dedicated to making your travel dreams a reality."},
    {"id": "116", "content": "Explore new cultures and cuisines with Contoso Travel's guided tours."},
    {"id": "117", "content": "Contoso Travel offers travel packages for families, couples, and solo travelers."},
    {"id": "118", "content": "Our travel advisors are experts in creating personalized travel experiences."},
    {"id": "119", "content": "Contoso Travel is proud to partner with local businesses and communities."},
    {"id": "120", "content": "Join our loyalty program for exclusive discounts and rewards."}
]

# Add documents to the index
search_client.upload_documents(documents)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],
    name="TravelAgent",
    instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
)

### Running the Agent with Streaming Invocation

The main asynchronous loop creates a thread for the conversation and, for each user input,  so that the agent sees the retrieval context. The user message is also added, and then the agent is invoked using streaming. The output is printed as it streams in.

In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


You should see output similar to the following:

```text
User: 
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```